In [215]:
from numpy import *
from scipy.special import erf
from scipy import optimize
from scipy.stats import norm,chi2
random.seed(1)

# P1 Given the following SDE (Vasicek model)
$$
\\
dr_t = \kappa(\overline{r} - r_t)dt + \sigma dW_t
\\
$$
with $r_0 = 5\%, \sigma = 10\%, \kappa = 0.82, \overline{r} = 5\%$

In [2]:
# set parameter for P1 Vasicek Model dynamics
param_P1 = {"r0": 0.05, "sigma": 0.10, "kappa": 0.82, "r_bar": 0.05}

## (a) Price ZCB using Monte Carlo Simulation at t = 0
### Monte Carlo Simulation with 50000 paths
With face value of \$1000, maturing in T = 0.5 years

In [34]:
def ZCB_Vasicek_MC(t, T, rt, simu_paths, **param):
    # Vasicek model parameters
#     r0 = param['r0']
    sigma = param['sigma']
    kappa = param['kappa']
    r_bar = param['r_bar']
    
    t_steps = int(floor(365*(T-t)))
    dt = T / float(t_steps)
    # mark the evaluation time step
#     at_time_step = int(floor(365*at_time)) + 1
    
    # stochastic part
    dW_t = sqrt(dt) * random.standard_normal((t_steps, simu_paths))
    
    # Vasicek dynamics to calculate short interest rate
    r = zeros((t_steps+1, simu_paths))
    r[0] = ones(simu_paths) * rt
    for i in range(t_steps):
        r[i+1] = r[i] + kappa*(r_bar - r[i])*dt + sigma*dW_t[i]
    
    # discount factor at the evaluation time
    R = - dt * sum(r[1:], axis = 0)
    
    value = mean(exp(R)) # dollar value at time t
    rT = r[-1]
    return value, R, rT

In [202]:
p1a,_,_ = ZCB_Vasicek_MC(0, 0.5, param_P1['r0'], 50000, **param_P1)
print('The bond price calculated by Monte Carlo simulation is', p1a*1000)

The bond price calculated by Monte Carlo simulation is 975.37700229


### Explicit Formula for Answer Check

In [105]:
def ZCB_Vasicek(t, T, rt, **param):
    # Vasicek model parameters
    A = factor_A(t, T, **param)
    B = factor_B(t, T, **param)
    P = A * exp(-B*rt)
    return P
def factor_B(t, T, **param):
    kappa = param['kappa']
    return (1 - exp(-kappa*(T-t)))/kappa
def factor_A(t, T, **param):
    sigma = param['sigma']
    kappa = param['kappa']
    r_bar = param['r_bar']
    A = exp((r_bar - sigma**2 / (2*kappa**2)) * \
        (factor_B(t, T, **param) - (T-t)) - sigma**2 / (4*kappa) * factor_B(t, T, **param)**2)
    return A

In [203]:
p1a_analytial = ZCB_Vasicek(0, 0.5, param_P1['r0'], **param_P1)
print('The bond price calculated by explicit formula is',p1a_analytial*1000)

The bond price calculated by explicit formula is 975.461028359


## (b) Price Coupon Paying Bond using Monte Carlo Simulation at t = 0
### Monte Carlo Simulation with 50000 paths
With face value of \$1000, semmiannual coupons of \$30, maturing in T = 4 years

In [175]:
def SemiCB_Vasicek_MC(coupon_rate, maturity, simu_paths, **param):
    # Vasicek model parameters
    r0 = param['r0']
    sigma = param['sigma']
    kappa = param['kappa']
    r_bar = param['r_bar']
    
    time_period = int(maturity * 2)
    
    T = linspace(0.5, maturity, time_period)
    coupons = append(ones(time_period-1) * coupon_rate, 1 + coupon_rate)
    
    values = asarray([])
    for T_ in T:
        # evaluate for each decomposed ZCB
        value, _, _ = ZCB_Vasicek_MC(0, T_, r0, simu_paths, **param)
        values = append(values, value)

    price = sum(multiply(values, coupons))
    return price

In [176]:
p1b = SemiCB_Vasicek_MC(0.03, 4, 50000, **param_P1)
print('The coupon paying bond price calculated by Monte Carlo simulation is', p1b*1000)

The coupon paying bond price calculated by Monte Carlo simulation is 1048.53296921


## (c) Price European Call option on ZCB in part (a) using Monte Carlo Simulation at t = 0
With option maturity of 3 monts and strike price K = \$980. Explicit formula for underlying bond price.


In [146]:
def ZCB_Euro_Call_Vasicek_MC(face_value, strike_price, bond_maturity, option_maturity, simu_paths, **param):
    
    # run monte carlo till option maturity time T, retrieve r_T and discount rate R_T
    _,R,r_strike = ZCB_Vasicek_MC(0, option_maturity, param['r0'], simu_paths, **param)
    
    # calculate value at strike using explicit method
    discount_strike = exp(R)
    
    # vectorize the calculation of explicit price
    vfunc = vectorize(ZCB_Vasicek)
    value_strike = vfunc(option_maturity, bond_maturity, r_strike, **param)
    
    values = maximum(value_strike*face_value - strike_price, 0)
    price = mean(multiply(values, discount_strike))
    return price

In [38]:
p1c = ZCB_Euro_Call_Vasicek_MC(1000, 980, 0.5, 0.25, 50000, **param_P1)
print('The price of a European Call option on the pure discount bond in part (a) is',p1c)

The price of a European Call option on the pure discount bond in part (a) is 8.84810336163


### Explicit Formula for Answer Check

In [158]:
def cnd(x):
    return (1.0/2)*(1+erf(x/sqrt(2)))

def d1(PtS, PtT, sigma_p, L, K):
    return log(L*PtS/K/PtT)/sigma_p + sigma_p / 2

def ZCB_Euro_Call_Vasicek_analytical(t, T, S, rt, L, K, **param):
    sigma = param['sigma']
    kappa = param['kappa']
    PtS = ZCB_Vasicek(t, S, rt, **param)
    PtT = ZCB_Vasicek(t, T, rt, **param)
    sigma_p = sqrt((1-exp(-2*kappa*(T-t)))/(2*kappa)) * \
             ((1-exp(-kappa*(S-T)))/kappa)*sigma
    c = L * PtS * cnd(d1(PtS, PtT, sigma_p, L, K)) - \
        K * PtT * cnd(d1(PtS, PtT, sigma_p, L, K) - sigma_p)
    return c

In [159]:
p1c_analytial = ZCB_Euro_Call_Vasicek_analytical(0, 0.25, 0.5, param_P1['r0'], 1000, 980, **param_P1)
print('The explicit solved price of a European Call option on the pure discount bond in part (a) is', p1c_analytial)

The explicit solved price of a European Call option on the pure discount bond in part (a) is 8.8833618642


## (d) Price European Call option on Coupon Paying Bond in part (b) using Monte Carlo Simulation at t = 0
With option maturity of 3 months and strike price K = $980. Monte Carlo simulation for underlying bond price.

In [182]:
def P1d(face_value, strike_price, bond_maturity, option_maturity,  \
        coupon, main_simu_paths, branch_simu_paths, **param):
    
    time_period = int(bond_maturity*2)
    
    T = linspace(0.5, bond_maturity, time_period)
    coupons = append(ones(time_period-1) * coupon, face_value + coupon)

    # 1 simulate to option maturity
    _, R, r_strike = ZCB_Vasicek_MC(0, option_maturity, param['r0'], main_simu_paths, **param)
    discount_strike = exp(R)
    
    # 2 use MC to price underlying bond
    path_payoffs = asarray([])
    for r_ in r_strike:
        values = asarray([])
        for T_i in T:
            value, _, _ = ZCB_Vasicek_MC(option_maturity, T_i, r_, branch_simu_paths, **param)
            values = append(values, value)
        path_value = sum(multiply(coupons,values))
        path_payoff = maximum(path_value-strike_price, 0)
        
        path_payoffs = append(path_payoffs, path_payoff)
        
    price = mean(multiply(path_payoffs, discount_strike))
    return price

In [214]:
P1d(1000, 980, 4, 0.25, 30, 2000, 200, **param_P1)

70.370510232438917

In [184]:
def P1e(face_value, strike_price, bond_maturity, option_maturity,  \
        coupon, main_simu_paths, **param):
    
    time_period = int(bond_maturity*2)
    
    T = linspace(0.5, bond_maturity, time_period)
    coupons = append(ones(time_period-1) * coupon, face_value + coupon)

    # 1 simulate to option maturity
    _, R, r_strike = ZCB_Vasicek_MC(0, option_maturity, param['r0'], main_simu_paths, **param)
    discount_strike = exp(R)
    
    # 2 use MC to price underlying bond
    path_payoffs = asarray([])
    vfunc = vectorize(ZCB_Vasicek)
    for r_ in r_strike:
        values = vfunc(option_maturity, T, r_, **param)
        path_value = sum(multiply(coupons,values))
        path_payoff = maximum(path_value-strike_price, 0)
        
        path_payoffs = append(path_payoffs, path_payoff)
        
    price = mean(multiply(path_payoffs, discount_strike))
    return price

In [213]:
P1e(1000, 980, 4, 0.25, 30, 2000, **param_P1)

82.50143783211368

In [209]:
T = linspace(0.5, 4, 8)
coupons = append(ones(7)*30, 1030)
vfuncA = vectorize(factor_A)
dA = vfuncA(0.25, T, **param_P1)
vfuncB = vectorize(factor_B)
dB = exp(-vfuncB(0.25, T, **param_P1))

def price_r_star(r_star):
    Ki = multiply(coupons, multiply(dA, power(dB, r_star)))
    return sum(Ki)-980

In [211]:
root = optimize.newton(price_r_star, 0.05)
vfunc3 = vectorize(ZCB_Vasicek)
KKi = vfunc3(0.25, T, root, **param_P1) * coupons
cbop = zeros(8)
for i in range(8):
    cbop[i] = ZCB_Euro_Call_Vasicek_analytical(0, 0.25, T[i], param_P1['r0'], coupons[i], KKi[i], **param_P1)

price_C = sum(cbop)
price_C

82.139018428844764

array([  29.15059153,   27.76824627,   26.68542697,   25.80057656,
         25.04782369,   24.38445974,   23.78275471,  797.38012053])

# P2 Given the following SDE (CIR model)
$$
\\
dr_t = \kappa(\overline{r} - r_t)dt + \sigma \sqrt{r_t} dW_t
\\
$$
with $r_0 = 5\%, \sigma = 12\%, \kappa = 0.92, \overline{r} = 5.5\%$

In [216]:
param_P2 = {"r0": 0.05, "sigma": 0.12, "kappa": 0.92, "r_bar":0.055}

In [217]:
def ZCB_CIR_MC(t, T, rt, simu_paths, **param):
    # Vasicek model parameters
#     r0 = param['r0']
    sigma = param['sigma']
    kappa = param['kappa']
    r_bar = param['r_bar']
    
    t_steps = int(floor(365*(T-t)))
    dt = T / float(t_steps)
    # mark the evaluation time step
#     at_time_step = int(floor(365*at_time)) + 1
    
    # stochastic part
    dW_t = sqrt(dt) * random.standard_normal((t_steps, simu_paths))
    
    # Vasicek dynamics to calculate short interest rate
    r = zeros((t_steps+1, simu_paths))
    r[0] = ones(simu_paths) * rt
    for i in range(t_steps):
        r[i+1] = r[i] + kappa*(r_bar - r[i])*dt + sigma*sqrt(r[i])*dW_t[i]
    
    # discount factor at the evaluation time
    R = - dt * sum(r[1:], axis = 0)
    
    value = mean(exp(R)) # dollar value at time t
    rT = r[-1]
    return value, R, rT

In [290]:
ZCB_CIR_MC(0, 1, param_P2['r0'], 50000, **param_P2)

(0.94966605717613273,
 array([-0.06577857, -0.05957871, -0.04104054, ..., -0.04567638,
        -0.05492506, -0.06966266]),
 array([ 0.05166581,  0.09363765,  0.0305247 , ...,  0.02226651,
         0.06032883,  0.06865502]))

In [293]:
def ZCB_CIR(t, T, rt, **param):
    r_bar = param['r_bar']
    kappa = param['kappa']
    sigma = param['sigma']
    
    h1 = sqrt(kappa**2 + 2 * sigma**2)
    h2 = (kappa + h1) / 2.
    h3 = (2 * kappa * r_bar) / sigma**2
    
    B = (exp(h1 * (T-t)) - 1) / (h2 * (exp(h1 * (T-t)) - 1) + h1)
    A = ((h1*exp(h2 * (T-t))) / (h2 * (exp(h1 * (T-t)) - 1) + h1))**h3
    P = A*exp(-B*rt)
    return P

In [292]:
ZCB_CIR(0, 1, 0.055, **param_P2)

0.94655127356327939

In [294]:
def ZCB_Euro_Call_CIR_MC(face_value, strike_price, bond_maturity, option_maturity, \
                         main_simu_paths, **param):
    
    # run monte carlo till option maturity time T, retrieve r_T and discount rate R_T
    _,R,r_strike = ZCB_CIR_MC(0, option_maturity, param['r0'], main_simu_paths, **param)
    
    # calculate value at strike using explicit method
    discount_strike = exp(R)
    
    # vectorize the calculation of explicit price
    vfunc = vectorize(ZCB_CIR)
    values = vfunc(option_maturity, bond_maturity, r_strike, **param)
#     values = asarray([])
#     for r_ in r_strike:
#         value,_,_ = ZCB_CIR_MC(option_maturity, bond_maturity, r_, branch_simu_paths, **param)
#         values = append(values, value)
        
    payoffs = maximum(values*face_value - strike_price, 0)
    price = mean(multiply(payoffs, discount_strike))
    return price

In [295]:
ZCB_Euro_Call_CIR_MC(1000, 980, 1, 0.5, 50000, **param_P2)

0.39625010263465765

In [244]:
def ZCB_Eurocall_CIR(T, S, L, K, **param):
    r0 = param['r0']
    r_bar = param['r_bar']
    kappa = param['kappa']
    sigma = param['sigma']
    
    
    P0S = ZCB_CIR(0, S, r0, **param)
    P0T = ZCB_CIR(0, T, r0, **param)
    
    h1 = sqrt(kappa**2 + 2 * sigma**2)
    h2 = (kappa + h1) / 2.
    h3 = (2 * kappa * r_bar) / sigma**2
    
    BTS = (exp(h1 * (S-T)) - 1) / (h2 * (exp(h1 * (S-T)) - 1) + h1)
    ATS = ((h1*exp(h2 * (S-T))) / (h2 * (exp(h1 * (S-T)) - 1) + h1))**h3
    
    theta = sqrt(kappa**2 + 2*sigma**2)
    phi = (2*theta) / (sigma**2 * (exp(theta*T) - 1))
    psi = (kappa + theta) / sigma**2
    rstar = log(ATS / K)/BTS
    
    
# # interest rate parameters:
# rzero = param_P2['r0']
# rbar = param_P2['r_bar']
# kappa = param_P2['kappa']
# sigma = param_P2['sigma']

# # Option parameters:        European Call
# OM = 0.5  # Option Maturity in years
# OBM = 1  # Option's Bond Maturity in years
# K = 0.98

# # Bond parameters:
# BM = 1 # Bond Maturity in years


# def PDB(r,M):
#     P = exp(-r*M)
#     return P

# # Pure discount Bond price calculations:
# def PBond(M,kappa,sigma,rbar,rzero):
#     phi1 = sqrt(kappa ** 2 + 2 * sigma ** 2)
#     phi2 = (kappa + phi1) / 2
#     phi3 = (2 * kappa * rbar) / sigma ** 2
#     B = (PDB(-phi1,M)-1)/(phi2*(PDB(-phi1,M)-1)+phi1)
#     A = ((phi1*PDB(-phi2,M))/(phi2*(PDB(-phi1,M)-1)+phi1))**phi3
#     P = A*PDB(B,rzero)
#     return P

# print('Price of the bond =',PBond(BM,kappa,sigma,rbar,rzero))

# def B(M,kappa,sigma):
#     phi1 = sqrt(kappa ** 2 + 2 * sigma ** 2)
#     phi2 = (kappa + phi1) / 2
#     B = (PDB(-phi1,M)-1)/(phi2*(PDB(-phi1,M)-1)+phi1)
#     return B

# def A(M,kappa,sigma,rbar):
#     phi1 = sqrt(kappa ** 2 + 2 * sigma ** 2)
#     phi2 = (kappa + phi1) / 2
#     phi3 = (2 * kappa * rbar) / sigma ** 2
#     A = ((phi1 * PDB(-phi2, M)) / (phi2 * (PDB(-phi1, M) - 1) + phi1)) ** phi3
#     return A

# sigmaR = ((sigma*sqrt(rzero))/BM)*B(BM,kappa,sigma) # volatility of the one-year spot rate
# POB = PBond(OBM,kappa,sigma,rbar,rzero)
# PO = PBond(OM,kappa,sigma,rbar,rzero)

# theta = sqrt((kappa**2) + 2*(sigma**2))
# phi = (2*theta)/((sigma**2)*(PDB(-theta,OM)-1))
# Y = (kappa+theta)/sigma**2
# BOB = B(OBM-OM,kappa,sigma)
# AOB = A(OBM-OM,kappa,sigma,rbar)

# d1 = 2*rzero*(phi+Y+BOB)
# d2 = (4*kappa*rbar)/sigma**2
# d3 = (2*(phi**2)*rzero*PDB(-theta,OM))/(phi+Y+BOB)
# d4 = 2*rzero*(phi+Y)
# d5 = (2*(phi**2)*rzero*PDB(-theta,OM))/(phi+Y)

# C = POB*chi2.cdf(d1,d2,d3)-K*PO*chi2.cdf(d4,d2,d5)
# print('call option of the bond',C)

Price of the bond = 0.949645807023
call option of the bond 0.00231586084505


$$
\begin{align}
dx_t &= -ax_t dt + \sigma dW_t^1 \\
dy_t &= -b y_t dt + \eta dW_t^2 \\
r_t &= x_t + y_t + \phi_t
\end{align}
$$

In [257]:
param_P3 = {'x0':0, 'y0':0, 'phi0':0.03, 'r0':0.03, 'rho':0.7, 'a':0.1, 'b':0.3, 'sigma':0.03, 'eta':0.08}

In [272]:
def ZCB_G2PP_MC(t, T, xt, yt, rt, simu_paths, **param):
#     x0 = param['x0']
#     y0 = param['y0']
    phi0 = param['phi0']
#     r0 = param['r0']
    rho = param['rho']
    a = param['a']
    b = param['b']
    sigma = param['sigma']
    eta = param['eta']
    
    t_span = T - t
    t_steps = int(floor(t_span*365))
    dt = t_span / t_steps
    
    z1 = random.standard_normal((t_steps, simu_paths))
    z2 = random.standard_normal((t_steps, simu_paths))
    dW1 = sqrt(dt) * z1
    dW2 = sqrt(dt) * (rho * z1 + sqrt(1 - rho**2) * z2)

    x = zeros((t_steps+1, simu_paths))
    y = zeros((t_steps+1, simu_paths))

    x[0] = ones(simu_paths)*xt
    y[0] = ones(simu_paths)*yt

    for i in range(t_steps):
        x[i+1] = x[i] - a*x[i]*dt + sigma*dW1[i]
        y[i+1] = y[i] - b*y[i]*dt + eta*dW2[i]
    r = x + y + phi0
    R = exp(-dt*sum(r[1:], axis = 0))
    value = mean(exp(-dt*sum(r[1:], axis = 0)))
    return value, R, r[-1], x[-1], y[-1]

In [288]:
def ZCB_G2PP_EuroPut_MC(face_value, strike_price, t, T, S, xt, yt, rt, main_simu_paths, branch_simu_paths, **param):
    _, R_strike, r_strike, x_strike, y_strike = ZCB_G2PP_MC(t, T, xt, yt, rt, main_simu_paths, **param)
    
    values = asarray([])
    for index, r_ in enumerate(r_strike):
        value,_,_,_,_ = ZCB_G2PP_MC(T, S, x_strike[index], y_strike[index], r_, branch_simu_paths, **param)
        values = append(values, value)
    payoffs = maximum(strike_price - multiply(values, face_value), 0)
    price = mean(multiply(R_strike, payoffs))
    return price
ZCB_G2PP_EuroPut_MC(1000, 950, 0, 0.5, 1., param_P3['x0'],param_P3['y0'],param_P3['r0'], 1000, 800, **param_P3)

1.721009090440454

In [281]:
def ZCB_G2PP(T, **param):
    x0 = param['x0']
    y0 = param['y0']
    phi0 = param['phi0']
    r0 = param['r0']
    rho = param['rho']
    a = param['a']
    b = param['b']
    sigma = param['sigma']
    eta = param['eta']
    
    V1 = sigma**2/a**2*(T+2/a*exp(-a*T)-exp(-2*a*T)/(2*a)-1.5/a)
    V2 = eta**2/b**2*(T+2/b*exp(-b*T)-exp(-2*b*T)/(2*b)-1.5/b)
    V3 = 2*rho*sigma*eta/(a*b)*(T+(exp(-a*T)-1)/a+(exp(-b*T)-1)/b-(exp(-(a+b)*T)-1)/(a+b))
    V0T = V1+V2+V3
    
    P1 = T*phi0
    P2 = (1-exp(-a*T))/a*x0
    P3 = (1-exp(-b*T))/b*y0
    P0T = exp(-P1-P2-P3+0.5*V0T)
    return P0T

def PP(coef, t, T):
    return (1 - exp(-coef*(T-t)))
def ZCB_G2PP_Europut(T, S, L, K, **param):
    rho = param['rho']
    a = param['a']
    b = param['b']
    sigma = param['sigma']
    eta = param['eta']
    
    s1 = sigma**2 / (2*a**3) * (PP(a, T, S))**2 * PP(2*a, 0, T)
    s2 = eta**2 / (2*b**3) * (PP(b, T, S))**2 * PP(2*b, 0, T)
    s3 = 2*rho * (sigma*eta)/(a*b*(a+b)) * PP(a, T, S) * PP(b, T, S) * PP(a+b, 0, T)
    
    big_sigma = sqrt(s1+s2+s3)
    
    P0S = ZCB_G2PP(S, **param)
    P0T = ZCB_G2PP(T, **param)
    
    d1 = (log(L*P0S / (K*P0T))) / big_sigma + 0.5 * big_sigma
    d2 = d1 - big_sigma
    
    price = -L*P0S*cnd(-d1) + P0T*K*cnd(-d2)
    return price

ZCB_G2PP_Europut(0.5, 1., 1000, 950, **param_P3)

1.8609598749597467